In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer_all = Timer()
timer_all.reset()

In [3]:
timer.reset()
tb = TbPacientes()
# tb.update()
timer.stop()

1.4180757999420166

In [4]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

1.1524431705474854

In [5]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (583786, 24)
casos tb_pacientes: (583786, 48)


In [6]:
casosc['identificacao'] = casosc['identificacao'].apply(int)
casosc = casosc.sort_values('identificacao')

In [7]:
casost['identificacao'] = casost['identificacao'].apply(int)
casost = casost.sort_values('identificacao')

In [8]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [9]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [10]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)
if (len(casos_confirmados_nao_tb_pacientes) > 0) or (len(tb_pacientes_nao_casos_confirmados) > 0):
    raise Exception("diferentes")

casos_confirmados_nao_tb_pacientes: (0, 24)
tb_pacientes_nao_casos_confirmados: (0, 48)


In [11]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [12]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [13]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['nome'].values == casost['nome'].values))

True
True
True


In [14]:
timer.reset()

# sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
# sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
# i = 0
# for (ibge_cc, df_ibge_cc), (ibge_tb, df_ibge_tb) in zip(casosc.groupby('ibge7'),casost.groupby('ibge_res_pr')):
#     print(i,end=' ')
#     i+=1
#     if ibge_cc != ibge_tb:
#         raise Exception(ibge_cc,ibge_tb)

#     df_ibge_cc = df_ibge_cc.sort_values('nome_hash')
#     df_ibge_tb = df_ibge_tb.sort_values('nome_hash')

#     duplicados_ibge_cc = df_ibge_cc.loc[ df_ibge_cc.duplicated('nome_hash', keep=False) ]
#     duplicados_ibge_tb = df_ibge_tb.loc[ df_ibge_tb.duplicated('nome_hash', keep=False) ]

#     sorted_casosc = sorted_casosc.append(df_ibge_cc.loc[~df_ibge_cc.index.isin(duplicados_ibge_cc.index)])
#     sorted_casost = sorted_casost.append(df_ibge_tb.loc[~df_ibge_tb.index.isin(duplicados_ibge_tb.index)])

#     for (nome_cc, df_nome_cc), (nome_tb, df_nome_tb) in zip(duplicados_ibge_cc.groupby('nome_hash'),duplicados_ibge_tb.groupby('nome_hash')):
#         if nome_cc != nome_tb:
#             raise Exception(nome_cc,nome_tb)

#         sorted_casosc = sorted_casosc.append(df_nome_cc.sort_values('idade'))
#         sorted_casost = sorted_casost.append(df_nome_tb.sort_values('idade'))


# print(f"casos casos_confirmados: {sorted_casosc.shape}")
# print(f"casos tb_pacientes: {casost.shape}")
# print(timer.stop())
# casosc = sorted_casosc
# casost = sorted_casost

In [15]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['nome'].values == casost['nome'].values))

True
True
True


In [16]:
casosc = casosc.rename(columns={'ibge7':'ibge_resid','nome':'paciente','dt_diag':'data_diagnostico','comunicacao':'data_comunicacao','is':'data_1o_sintomas'})

# casosc = casosc.drop(columns=['data_com_obito'])

casosc['ordem'] = casosc['ordem'].apply(int)
casosc['ibge_atend'] = casost['ibge_atend_pr'].apply(int).values.tolist()
casosc['ibge_resid'] = casosc['ibge_resid'].apply(int)

casosc.loc[casosc['rs'].isnull(),'rs'] = 99
casosc['rs'] = casosc['rs'].apply(int)

casosc['recuperado'] = [ 1 if recuperado=="Recuperado" else 0 for recuperado in casost['status'] ]
casosc['obito'] = [ 1 if obito=="SIM" else 0 for obito in casosc['obito'] ]
# casosc.loc[casosc['obito']==1, 'data_com_obito'] = casost.loc[casost['obito']=="SIM", 'dt_com_obito']

casosc = pd.merge(casosc,casost[['identificacao','dt_com_obito']], how='left', on='identificacao')

casosc = casosc.rename(columns={'dt_com_obito':'data_com_obito'})

casosc['ativo'] = 0
casosc.loc[(casosc['recuperado']==0) & (casosc['obito']==0 ), 'ativo'] = 1
casosc.loc[(casosc['recuperado']==1) & (casosc['obito']==1 ), 'recuperado'] = 0

casosc = casosc[['ordem', 'identificacao', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'obito', 'data_obito', 'data_com_obito', 'recuperado', 'ativo', 'excluir']].sort_values('ordem')

KeyError: "['data_com_obito'] not in index"

In [ ]:
writer = pd.ExcelWriter(join(output,"casos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc.to_excel(writer,'geral',index=None)
worksheet = writer.sheets[f"geral"]
auto_fit_columns(worksheet,casosc)

writer.save()
writer.close()

In [ ]:
print(timedelta(seconds=timer_all.stop()))

In [ ]:
casosc

In [ ]:
casost